In [33]:
import numpy as np
import tensorflow as tf
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

ImportError: dlopen(/Users/atiemoasare/Projects/thesis/venv/lib/python3.8/site-packages/tensorflow/python/platform/_pywrap_cpu_feature_guard.so, 0x0002): Library not loaded: @rpath/libtensorflow_cc.2.dylib
  Referenced from: <A8C24B75-9F46-3078-83B4-DA46F5E64516> /Users/atiemoasare/Projects/thesis/venv/lib/python3.8/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so
  Reason: tried: '/Users/atiemoasare/Projects/thesis/venv/lib/python3.8/site-packages/tensorflow/python/../../_solib_darwin_arm64/libtensorflow_cc.2.dylib' (no such file), '/Users/atiemoasare/Projects/thesis/venv/lib/python3.8/site-packages/tensorflow/python/../../_solib_darwin_arm64/_U_S_Stensorflow_Spython_C_Upywrap_Utensorflow_Uinternal.so_Ucclib___Utensorflow/libtensorflow_cc.2.dylib' (no such file), '/Users/atiemoasare/Projects/thesis/venv/lib/python3.8/site-packages/tensorflow/python/libtensorflow_cc.2.dylib' (no such file), '/Users/atiemoasare/Projects/thesis/venv/lib/python3.8/site-packages/tensorflow/py

In [ ]:
def get_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    return model

In [30]:
def encrypt_message(message, recipient_public_key):
    # Step 1: Generate ephemeral key pair
    ephemeral_private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    ephemeral_public_key = ephemeral_private_key.public_key()

    # Step 2: Derive shared secret
    shared_secret = ephemeral_private_key.exchange(ec.ECDH(), recipient_public_key)

    # Step 3: Derive encryption key and nonce
    derived_key_material = HKDF(
        algorithm=hashes.SHA256(),
        length=32 + 12,  # AES key length + nonce length
        salt=None,
        info=b'',
    ).derive(shared_secret)

    encryption_key = derived_key_material[:32]
    nonce = derived_key_material[32:]

    # Step 4: Encrypt the message
    cipher = AESGCM(encryption_key)
    ciphertext = cipher.encrypt(nonce, message.encode(), None)

    # Step 5: Return the ephemeral public key and ciphertext
    return ephemeral_public_key, ciphertext


def decrypt_message(ciphertext, ephemeral_public_key, recipient_private_key):
    # Step 1: Derive shared secret using recipient's private key and ephemeral public key
    shared_secret = recipient_private_key.exchange(ec.ECDH(), ephemeral_public_key)

    # Step 2: Derive encryption key and nonce
    derived_key_material = HKDF(
        algorithm=hashes.SHA256(),
        length=32 + 12,  # AES key length + nonce length
        salt=None,
        info=b'',
    ).derive(shared_secret)

    encryption_key = derived_key_material[:32]
    nonce = derived_key_material[32:]

    # Step 3: Decrypt the ciphertext
    cipher = AESGCM(encryption_key)
    decrypted_message = cipher.decrypt(nonce, ciphertext, None)

    # Step 4: Return the decrypted message
    return decrypted_message.decode()


recipient_private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
recipient_public_key = recipient_private_key.public_key()


message_to_encrypt = "Hello, how are you doing?"

# recipient_public_key = ec.EllipticCurvePublicKey.from_encoded_point(ec.SECP256R1(), recipient_public_key_bytes)
ephemeral_public_key, ciphertext = encrypt_message(message_to_encrypt, recipient_public_key)

ephemeral_public_key_bytes = ephemeral_public_key.public_bytes(
    encoding=serialization.Encoding.X962,
    format=serialization.PublicFormat.CompressedPoint,
)
ciphertext_hex = ciphertext.hex()

print("Ephemeral Public Key:", ephemeral_public_key_bytes)
print("Ciphertext:", ciphertext_hex)


decrypted_message = decrypt_message(ciphertext, ephemeral_public_key, recipient_private_key)
print("Decrypted Message:", decrypted_message)

Ephemeral Public Key: b'\x02\xab\x08\xc4W\x98=}\xba\xa9P\x0fA\xe6\x85\xed\x16\xf7J\xb9\x8e\xa1Q[>\xf4S\xeb\x81\x12\xfd\xed\x8b'
Ciphertext: 93b65d4974c28f97d2253fba8abe699fee808e3b828311aa53b4d8327723e81763a27b59d8cf9d2a05
Decrypted Message: Hello, how are you doing?
